# Fixing data
Here is the process of data cleaning 

In [23]:
import pandas as pd
import re

In [24]:

df = pd.read_csv("eurocup_2020_results.csv")
df2 = pd.read_csv('players2021_complete.csv')


In [73]:
df.head()

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,11.07.2021,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"[{'event_team': 'away', 'event_time': "" 2' "", ...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
1,Semi-finals,07.07.2021,False,False,False,England,Denmark,2,1,59%,41%,20,6,10,3,50%,50%,"[{'event_team': 'away', 'event_time': "" 30' "",...","[{'Player_Name': 'Kane', 'Player_Number': '9'}...","[{'Player_Name': 'Krogh Damsgaard', 'Player_Nu..."
2,Semi-finals,06.07.2021,True,4,2,Italy,Spain,1,1,29%,71%,7,16,4,5,49%,51%,"[{'event_team': 'away', 'event_time': "" 51' "",...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Torres', 'Player_Number': '1..."
3,Quarter-finals,03.07.2021,False,False,False,Ukraine,England,0,4,48%,52%,7,10,2,6,42%,59%,"[{'event_team': 'away', 'event_time': "" 4' "", ...","[{'Player_Name': 'Yaremchuk', 'Player_Number':...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
4,Quarter-finals,03.07.2021,False,False,False,Czech Republic,Denmark,1,2,56%,44%,16,11,5,7,48%,52%,"[{'event_team': 'away', 'event_time': "" 5' "", ...","[{'Player_Name': 'Schick', 'Player_Number': '1...","[{'Player_Name': 'Dolberg', 'Player_Number': '..."


# working with df2: individual players  -  deprecated-> done through webscraping

In [74]:
#we will delete some unwanted columns with df.drop(columns=['A', 'B'], inplace=True)
#df2.drop(columns=['hits','potential'], inplace=True)
df2.head()

,player_id,name,nationality,position,overall,age,hits,potential,team
0,158023,Lionel Messi,Argentina,ST|CF|RW,94,33,299,94,FC Barcelona
1,20801,Cristiano Ronaldo,Portugal,ST|LW,93,35,276,93,Juventus
2,190871,Neymar Jr,Brazil,CAM|LW,92,28,186,92,Paris Saint-Germain
3,203376,Virgil van Dijk,Netherlands,CB,91,29,127,92,Liverpool
4,200389,Jan Oblak,Slovenia,GK,91,27,47,93,Atlético Madrid


In [75]:
#to do: create a loop to eliminate all nationalities not present in Euro21
euroteams=['Austria', 'Belgium', 'Croatia', 'Czech Republic', 'Denmark', 'England', 'Finland', 'France', 'Germany', 'Hungary', 'Italy', 'Netherlands', 'North Macedonia', 'Poland', 'Portugal', 'Russia', 'Scotland', 'Slovakia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine', 'Wales']
eurostr=str(euroteams)
print(type(eurostr))
print(type(euroteams))


<class 'str'>
<class 'list'>


In [76]:
df2.shape

(17981, 9)

In [77]:
unique_nationalities=df2['nationality'].unique()
len(unique_nationalities)

162

# working with df1: match stats

## tests
We will attempt to just keep the data about goal scoring using different methods

In [ ]:
ev_test=df['events_list']

ev_test[2]  #10

In [ ]:
a=ev_test[2]
a=a.split(sep='{')
print (type(a))

#to do: eliminate all info that isnt a goal or own goal, perhaps change info to say name of team instead of just away and home

In [ ]:
a

In [ ]:
# #working as intended
# for data in a:
#     if 'Goal' in data:
#         print (data,'\n')
#     if 'goal' in data:
#           print (data,'\n')

In [ ]:
#[i for i in my_list if '91' not in i and '18' not in i]
b=[data for data in a if 'goal' in data.lower() or 'Penalty' in data and 'PK' not in data and 'Disallowed' not in data]
c=[data for data in b if 'False' not in data]


In [ ]:
#x = txt.replace("thingtoremove", "subs")
#re.sub(r'\W+', '', your_string)
team=[]
minute=[]
player=[]
for data in b:
    team.append(re.findall(('away|home'),data))
    minute.append(re.search('\d+',data).group())
    if 'Own goal' in data:
        player.append(re.search('(?:action_player_1).*?\w+\s\w*',data).group().replace("action_player_1': ' ","own goal-"))
    else:
        player.append(re.search('(?:action_player_1).*?\w+\s\w*',data).group().replace("action_player_1': ' ",""))

team=[x for [x] in team]
print(type(team))
print(team)
print(minute)
print(player)

#re.sub(r'[^A-Za-z0-9 ]+', '', s)

In [ ]:
final=[(team[i],player[i],minute[i]) for i in range (len(player))]
final

In [ ]:
final_l=[]
for data in final:
    
    final_l.append(list(data))
    
final_l

## function defining and  applying


In [39]:
def pichichi(event):
    team=[]
    minute=[]
    player=[]
    final=[]
    try:
        event_=event.split(sep='{')
        #print (event_)

        event_2=[data for data in event_ if 'goal' in data.lower() or 'Penalty' in data and 'PK' not in data and 'Disallowed' not in data]
        event_2=[data for data in event_2 if 'False' not in data]
        event_2=[data for data in event_2 if 'Disallowed' not in data]
        #print(event_2)

        for data in event_2:        
            team.append(re.findall(('away|home'),data))

            minute.append(re.search('\d+',data).group())

            if 'Own goal' in data:
                player.append(re.search('(?:action_player_1).*?\w+\s\w*',data).group().replace("action_player_1': ' ","own goal-"))
            else:
                player.append(re.search('(?:action_player_1).*?\w+\s\w*',data).group().replace("action_player_1': ' ",""))

        team=[x for [x] in team]   
        final=[({'team':team[i]},{'player':player[i]},{'minute':minute[i]}) for i in range (len(player))]
        final_l=[]
        for data in final:
            final_l.append(list(data))
        if final_l==[]:
            final_l=['no goals']
        return final_l
    except:
        return 'error'
    



In [40]:
df = pd.read_csv("eurocup_2020_results.csv")

In [41]:
df['events_list'].sample(n=1)

29    [{'event_team': 'away', 'event_time': " 10' ",...
Name: events_list, dtype: object

###### apply f(x)

In [42]:
df['events_list']=df['events_list'].apply(pichichi)

###### after some battling, the function for filtering just the goals made during the match time (excluidng OT penalty shootout) is working as intended

In [43]:
df.head(n=1)

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,11.07.2021,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"[[{'team': 'away'}, {'player': 'Luke Shaw'}, {...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."


In [44]:
df.drop(columns=['lineup_home','lineup_away'], inplace=True)

In [45]:
df=df[['stage','date','team_name_home','team_home_score','team_name_away','team_away_score','events_list' ,'pens','pens_home_score','pens_away_score','possession_home','possession_away', 'total_shots_home','total_shots_away','shots_on_target_home','shots_on_target_away']]

In [46]:
df=df.rename(columns={'events_list':'goals_scorers'})

Some different json files will be created to assess which one fits the APIs purposes better 
- table wins


In [47]:
# df_json1=df.to_json()
df_json2=df.to_json(orient="table")
# df_json3=df.to_json(orient="columns")
# df_json4=df.to_json(orient="index")
# df_json5=df.to_json(orient="records")

In [48]:
# f= open('matches1.json', 'w')
# f.write(df_json)
# f.close()

f2= open('matches.json', 'w')
f2.write(df_json2)
f2.close()

# f3= open('matches3_col.json', 'w')
# f3.write(df_json3)
# f3.close()

# f4= open('matches4_index.json', 'w')
# f4.write(df_json4)
# f4.close()

# f5= open('matches5_rec.json', 'w')
# f5.write(df_json5)
# f5.close()

In [ ]:
#legacy function, just in case

def pichichi(event):
    team=[]
    minute=[]
    player=[]
    final=[]
    try:
        event_=event.split(sep='{')
        #print (event_)

        event_2=[data for data in event_ if 'goal' in data.lower() or 'Penalty' in data and 'PK' not in data and 'Disallowed' not in data]
        event_2=[data for data in event_2 if 'False' not in data]
        event_2=[data for data in event_2 if 'Disallowed' not in data]
        #print(event_2)

        for data in event_2:        
            team.append(re.findall(('away|home'),data))

            minute.append(re.search('\d+',data).group())

            if 'Own goal' in data:
                player.append(re.search('(?:action_player_1).*?\w+\s\w*',data).group().replace("action_player_1': ' ","own goal-"))
            else:
                player.append(re.search('(?:action_player_1).*?\w+\s\w*',data).group().replace("action_player_1': ' ",""))

        team=[x for [x] in team]   
        final=[(team[i],player[i],minute[i]) for i in range (len(player))]
        final_l=[]
        for data in final:
            final_l.append(list(data))
        if final_l==[]:
            final_l=['no goals']
        return final_l
    except:
        return 'error'
    